In [9]:
!pip install opacus
import torch
import random
import warnings
import numpy as np

warnings.filterwarnings("ignore")

# Seed
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# Set the device
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Running on {DEVICE}.')

Running on cuda:0.


In [10]:
from torchvision.datasets import USPS
DATA_ROOTU = '../usps'


# 7291 training data images
train_datasetUS = USPS(root=DATA_ROOTU, 
    train=True,
    download=True)

#upscale to 32x32
upscaled = []
for im in train_datasetUS.data:
  new1  = np.repeat(im, 2, axis=1)
  new2 = np.repeat(new1, 2, axis=0)
  upscaled.append(new2)
upscaledTrain = np.array(upscaled)

# flatten the train images
n_samples = len(upscaledTrain)
X_train = upscaledTrain.reshape((n_samples, -1))
y_train = np.array(train_datasetUS.targets)

X_trainU1 = []
y_trainU1 = []
#randomly selecting 1000 train images
random_list = random.sample(range(0, 7291), k=500)
for i, idx in enumerate(random_list):
  X_trainU1.append(X_train[idx])
  y_trainU1.append(y_train[idx])

X_trainU1 = torch.from_numpy(np.array(X_trainU1).astype(np.float32))
y_trainU1 = torch.from_numpy(np.array(y_trainU1).astype(np.int64))

In [11]:
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

DATA_ROOT = '../mnist'

# 60_000 training data images
train_datasetM = MNIST(root=DATA_ROOT, 
    train=True, 
    transform=transforms.ToTensor(), 
    download=True)
#scaling image to 32x32
base = train_datasetM.data
expand1 = torch.cat([base, torch.zeros(60000, 28, 4)], 2)
expandX_train = torch.cat([expand1, torch.zeros(60000, 4, 32)], 1)
# flatten the train images
n_samples = len(expandX_train)
X_trainM = expandX_train.reshape((n_samples, -1))
y_trainM = train_datasetM.targets

X_testM1 = []
y_testM1 = []
#randomly selecting 10_000 test images
random_list = random.sample(range(0, 60_000), k=1000)
for i, idx in enumerate(random_list):
  X_testM1.append(X_trainM[idx].numpy())
  y_testM1.append(y_trainM[idx].numpy())

X_testM1 = torch.from_numpy(np.array(X_testM1).astype(np.float32))
y_testM1 = torch.from_numpy(np.array(y_testM1).astype(np.int64))

In [12]:
from torch.utils.data import Dataset

class DataSet(Dataset):
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y
    if len(self.X) != len(self.Y):
      raise Exception("The length of X does not match the length of Y")

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    _x = self.X[index]
    _y = self.Y[index]

    return _x, _y

In [13]:
import torch
import numpy as np
from torch.nn import Module, CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from opacus import PrivacyEngine

BATCH_SIZE = 50
LEARNING_RATE = 0.01
MAX_EPOCHS = 7
DEFAULT_LOSS = CrossEntropyLoss() 
CLIP = 1.0
DELTA = 1e-3
EPSILON = 10.0
SECURE_MODE = False

class FitModule(Module):
    def fit(self,
            X_train,
            y_train,
            batch_size=BATCH_SIZE,
            lr=LEARNING_RATE,
            loss_criteria=DEFAULT_LOSS,
            device=DEVICE,
            tmc=False):
        """fits the model with the train data, evaluates model during training on validation data
        # Arguments
            X_train: training data array of images
            y_train: training data array of labels
            lr: learning rate
            batch_size: number of samples per gradient update
            loss_criteria: training loss
            device: device used, GPU or CPU
            tmc: if fit is called within TMC 
        """
        self.train()
        if(not torch.is_tensor(X_train)):
          X_train = torch.from_numpy(X_train.astype(np.float32))
          y_train = torch.from_numpy(y_train.astype(np.int64))
        # create dataloader
        train_dataloader = DataLoader(DataSet(X_train, y_train), batch_size, shuffle=False)

      
        optimizer = Adam(self.parameters(), lr=lr)

        privacy_engine = PrivacyEngine(secure_mode=SECURE_MODE)


        model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon( 
            module=self,
            optimizer=optimizer,
            data_loader=train_dataloader,
            epochs=MAX_EPOCHS,
            target_epsilon=EPSILON,
            target_delta=DELTA,
            max_grad_norm=CLIP
        )


        losses = []
        for epoch in range(MAX_EPOCHS):
            # TRAINING
            self.train()
            train_epoch_loss = 0
            # run batches
            for i, (X, Y) in enumerate(train_dataloader):
                
                # move images, labels to device (GPU)
                X = X.to(device)
                Y = Y.to(device)

                # clear previous gradient
                optimizer.zero_grad()

                # feed forward the model
                output = model(X)
                train_batch_loss = loss_criteria(output, Y)
                
                # back propagation
                train_batch_loss.backward()
                
                # update parameters
                optimizer.step()
                

                # update training loss after each batch
                train_epoch_loss += train_batch_loss.item()


                # measure accuracy and record loss
                losses.append(train_batch_loss.item())
           
            epsilon = privacy_engine.get_epsilon(DELTA)           
            if not tmc: 
                print(
                    f"\tTrain Epoch: {epoch} \t"
                    f"Loss: {np.mean(losses):.6f} "
                    f"(ε = {epsilon:.2f}, δ = {DELTA})"
                )
            else:
                '''
                if epoch == MAX_EPOCHS - 1:
                    print(
                    f"\tTrain Epoch: {epoch} \t"
                    f"Loss: {np.mean(losses):.6f} "
                    )
                else:
                    '''
                continue
        model.remove_hooks() 

    def fitDL(self,
            dataloader,
            lr=LEARNING_RATE,
            loss_criteria=DEFAULT_LOSS,
            device=DEVICE
            ):
        """fits the model with the train data, evaluates model during training on validation data
        # Arguments
            dataloader: training data
            lr: learning rate
            batch_size: number of samples per gradient update
            loss_criteria: training loss
            device: device used, GPU or CPU
            tmc: if fit is called within TMC 
        """

        # create dataloader
        train_dataloader = dataloader

        optimizer = Adam(self.parameters(), lr=lr)

        accs = []
        losses = []
        for epoch in range(MAX_EPOCHS):
            # TRAINING
            self.train()
            train_epoch_loss = 0
            # run batches
            for i, (X, Y) in enumerate(train_dataloader):
                
                # move images, labels to device (GPU)
                X = X.to(device)
                Y = Y.to(device)

                # clear previous gradient
                optimizer.zero_grad()

                # feed forward the model
                output = self(X)
                train_batch_loss = loss_criteria(output, Y)
                
                # back propagation
                train_batch_loss.backward()
                
                # update parameters
                optimizer.step()
                
                # update training loss after each batch
                train_epoch_loss += train_batch_loss.item()

                preds = np.argmax(output.detach().cpu().numpy(), axis=1)
                labels = Y.detach().cpu().numpy()

                # measure accuracy and record loss
                acc = (preds == labels).mean()
                losses.append(train_batch_loss.item())
                accs.append(acc)
           
            # calculate training loss
            if epoch == MAX_EPOCHS-1:
                print(
                f"\tTrain Epoch: {epoch} \t"
                f"Loss: {np.mean(losses):.6f} "
                f"Accuracy: {np.mean(accs):.6f} "
                )
            else:
                continue

    def evaluate(self,
                 X_test,
                 y_test,
                 device=DEVICE):
        """evaluates performance of ML predictor on test data
        # Arguments
            X_test: test data array of images
            y_test: test data array of labels
            batch_size: number of samples per gradient update
            device: device used, GPU or CPU
        # Returns
            performane score"""
        self.eval()
        X_test = X_test.to(device)
        y_test = y_test.to(device)
        # TESTING
        test_predictions = torch.argmax(torch.softmax(self(X_test), 1), axis=1)
        test_accuracy = float(sum(test_predictions == y_test)) / y_test.shape[0]

        return test_accuracy

In [14]:
import torch.nn as nn

class LogisticRegression(FitModule):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, y):
        return self.linear(y)
        
    def score(self, X, y):
        return self.evaluate(X, y)
        
def get_model(input_dim, output_dim):
  return LogisticRegression(input_dim, output_dim).to(DEVICE)

In [15]:
import matplotlib
matplotlib.use('Agg')

class DShap(object):

    def __init__(self, X_train, y_train, X_test, y_test, metric, sources=None, seed=None, input_dim=1, output_dim=1):
        """
        Args:
            X_train: Train covariates
            y_train: Train labels
            X_test: Test covariates
            y_test: Test labels
            model_family: The model family used for learning algorithm
            metric: Evaluation metric
            X_val: Validation covariates
            y_val: Validation labels
            X_train_deep: Train deep features
            X_test_deep: Test deep features
            sources: An array or dictionary assigning each point to its group.
                If None, evey points gets its individual value.
            directory: Directory to save results and figures.
            seed: Random seed. When running parallel monte-carlo samples,
                we initialize each with a different seed to prevent getting
                same permutations.
        """

        if seed is not None:
            np.random.seed(seed)
        self.metric = metric
        self.input_dim = input_dim
        self.output_dim = output_dim
        self._initialize_instance(X_train, y_train, X_test, y_test, sources)
        self.model = get_model(self.input_dim, self.output_dim)
        self.random_score = self.init_score(self.metric)
        self.initial_state_dict = self.model.state_dict()
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"

    def _initialize_instance(self, X_train, y_train,X_test, y_test, sources=None):
        """loads or creates data"""

        sources = {i: np.array([i]) for i in range(X_train.shape[0])}
        self.sources = sources
        self.X_train, self.y_train = X_train, y_train
        self.X_test, self.y_test = X_test, y_test
        self.mem_tmc = np.zeros((0, self.X_train.shape[0]))
        idxs_shape = (0, self.X_train.shape[0] if self.sources is None else len(self.sources.keys()))
        self.idxs_tmc = np.zeros(idxs_shape).astype(int)
        self.vals_tmc = np.zeros((self.X_train.shape[0],))

    def init_score(self, metric):
        """ gives the value of an initial untrained model"""
        if metric == 'accuracy':
            return np.max(np.bincount(self.y_test).astype(float) / len(self.y_test))
        else:
            print('Invalid metric!')


    def run(self, iterations, tolerance=0.1, tmc_run=False):
        """calculates data sources(points) values
        Args:
            iterations: Number of iterations to run.
            err: stopping criteria for each of TMC-Shapley or G-Shapley algorithm.
            tolerance: Truncation tolerance. If None, the instance computes its own.
        """

        self.restart_model()
        self.model.fit(self.X_train, self.y_train)
        
        print('-----Starting TMC-Shapley calculations:')
        self._tol_mean_score()
        marginals, idxs = [], []
        
        for iteration in range(iterations):
            if 1000 * (iteration + 1) / iterations % 1 == 0:
                print('{} out of {} TMC_Shapley iterations.'.format(iteration + 1, iterations))
            marginals, idxs = self.one_iteration(tolerance=tolerance)
            self.mem_tmc = np.concatenate([self.mem_tmc, np.reshape(marginals, (1, -1))])
            self.idxs_tmc = np.concatenate([self.idxs_tmc, np.reshape(idxs, (1, -1))])
        self.vals_tmc = np.mean(self.mem_tmc, 0)
        print('-----TMC-Shapley values calculated!')

    def _tol_mean_score(self):
        """computes the average performance and its error using bagging"""
        scores = []
        self.restart_model()
        for _ in range(1):
            self.model.fit(self.X_train, self.y_train)
            for i in range(100):
                bag_idxs = np.random.choice(len(self.y_test), len(self.y_test))
                scores.append(self.model.score(self.X_test[bag_idxs], self.y_test[bag_idxs]))          
        self.mean_score = np.mean(scores)
        print("Average performance: ", self.mean_score)

    def one_iteration(self, tolerance):
        """runs one iteration of TMC-Shapley algorithm"""
        sources= self.sources
        idxs, marginal_contribs = np.random.permutation(len(sources.keys())), np.zeros(self.X_train.shape[0])
        new_score = self.random_score
        X_batch, y_batch = (np.zeros((0,) + tuple(self.X_train.shape[1:])), np.zeros((0,) + tuple(self.y_train.shape[1:])))
        
        truncation_counter = 0
        
        for n, idx in enumerate(idxs):
            old_score = new_score
            X_batch = np.concatenate((X_batch, self.X_train[sources[idx]]))
            y_batch = np.concatenate([y_batch, self.y_train[sources[idx]]])               
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                bool = True  # self.is_regression or len(set(y_batch)) == len(set(self.y_test))
                if bool:
                    self.restart_model()
                    self.model.fit(X_batch, y_batch, tmc=True)
                    new_score = self.model.score(self.X_test, self.y_test)
            marginal_contribs[sources[idx]] = (new_score - old_score) / len(sources[idx])
            
            if np.abs(new_score - self.mean_score) <= tolerance * self.mean_score:
                truncation_counter += 1
                if truncation_counter > 5:
                    break
            else:
                truncation_counter = 0
        return marginal_contribs, idxs

    def restart_model(self):
        self.model.load_state_dict(self.initial_state_dict)

In [16]:
import time

n_samples, n_features = X_train.shape

# parameters
metric = 'accuracy'
input_dim = n_features
iterations = 500
tolerance = 0.1

print('--- calculate tmc run time')
start_time = time.time()
dshap = DShap(X_trainU1, y_trainU1, X_testM1, y_testM1, metric, seed=seed, input_dim=input_dim, output_dim=10)
dshap.run(iterations, tolerance, tmc_run=True)

print("--- %s seconds ---" % ((time.time() - start_time)))
performances={}
values = dshap.vals_tmc
for idx in range(len(values)):
    performances[idx] = values[idx]

print("# Shapley Values: ", len(values), "\n Values: ", values)
print("Performances: ", performances)
time = time.time() - start_time
print("--- %s seconds ---" % (time))

#reverse performances
performancesRes = dict(reversed(list(performances.items())))

X_trainUP, y_trainUP = X_trainU1, y_trainU1

#deleting all negavite valued elements
for key in performancesRes.keys():
    if performancesRes[key] < 0:
        X_trainUP = np.delete(X_trainUP, key, axis=0)
        y_trainUP = np.delete(y_trainUP, key, axis=0)


#10_000 test data images
test_datasetM = MNIST(root=DATA_ROOT, 
    train=False, 
    transform=transforms.ToTensor(),
    download=True)

#scaling image to 32x32
base = test_datasetM.data
expand1 = torch.cat([base, torch.zeros(10000, 28, 4)], 2)
expandX_test = torch.cat([expand1, torch.zeros(10000, 4, 32)], 1)

# flatten the test images
n_samplesTest = len(expandX_test)
X_testM = expandX_test.reshape((n_samplesTest, -1))
y_testM = test_datasetM.targets

X_testM2 = []
y_testM2 = []
#randomly selecting 10_000 test images
random_list = random.sample(range(0, 10_000), k=1000)
for i, idx in enumerate(random_list):
  X_testM2.append(X_testM[idx].numpy())
  y_testM2.append(y_testM[idx].numpy())

X_testM2 = torch.from_numpy(np.array(X_testM2).astype(np.float32))
y_testM2 = torch.from_numpy(np.array(y_testM2).astype(np.int64))


#initial performance
model = get_model(input_dim, 10)
model.fitDL(DataLoader(DataSet(X_trainU1, y_trainU1), batch_size=BATCH_SIZE, shuffle=False))
initialPerformance = model.evaluate(X_testM2, y_testM2)
print("Initial performance: ", initialPerformance)

#testing the model on the test images
print(X_trainUP.shape, y_trainUP.shape)
model = get_model(input_dim, 10)
model.fitDL(DataLoader(DataSet(X_trainUP, y_trainUP), batch_size=BATCH_SIZE, shuffle=False))
newPerformance = model.evaluate(X_testM2, y_testM2)
print("New performance: ", newPerformance)

--- calculate tmc run time
	Train Epoch: 0 	Loss: 72.542194 (ε = 4.56, δ = 0.001)
	Train Epoch: 1 	Loss: 51.752268 (ε = 5.83, δ = 0.001)
	Train Epoch: 2 	Loss: 43.308241 (ε = 6.84, δ = 0.001)
	Train Epoch: 3 	Loss: 38.400357 (ε = 7.73, δ = 0.001)
	Train Epoch: 4 	Loss: 36.431263 (ε = 8.54, δ = 0.001)
	Train Epoch: 5 	Loss: 34.981608 (ε = 9.28, δ = 0.001)
	Train Epoch: 6 	Loss: 33.463244 (ε = 10.00, δ = 0.001)
-----Starting TMC-Shapley calculations:
	Train Epoch: 0 	Loss: 26.986891 (ε = 4.56, δ = 0.001)
	Train Epoch: 1 	Loss: 30.953870 (ε = 5.83, δ = 0.001)
	Train Epoch: 2 	Loss: 31.594042 (ε = 6.84, δ = 0.001)
	Train Epoch: 3 	Loss: 29.693511 (ε = 7.73, δ = 0.001)
	Train Epoch: 4 	Loss: 28.412986 (ε = 8.54, δ = 0.001)
	Train Epoch: 5 	Loss: 27.300721 (ε = 9.28, δ = 0.001)
	Train Epoch: 6 	Loss: 26.853031 (ε = 10.00, δ = 0.001)
Average performance:  0.15416999999999997
1 out of 500 TMC_Shapley iterations.
2 out of 500 TMC_Shapley iterations.
3 out of 500 TMC_Shapley iterations.
4 out of